#### Files needed for undulation

- regularGLL.txt, from `SE_MODEL.cpp` and `Quad.cpp`
- fCoeffs.txt, from `Undulation.cpp`
- undulatedElementTags.txt, from `Undulation.cpp`

#### For models with no undulation, please also follow the same procedure for regularGLL.txt, which will be the only file created

### Notes for creating the files above:
Drag the input folder of a specific model to the `build` directory of the `dev_axisem3d` folder. Run the build there.

#### This is a spherical mesh. Processing is different from Cartesian mesh.

In [ ]:
import numpy as np
import netCDF4 as nc
import re
import pyvtk
import pyvista as pv
import pickle

-------------

## Input parameters

### File locations

In [ ]:
# fileDir requires '/' at the end
baseDir = ''
modelName = ''
fileDir =baseDir+modelName+'/'
destDir = fileDir

### Physical slice

In [ ]:
phi_degree = 45

### Undulation

In [ ]:
undulated = True
### Set Nr as used in simulation
Nr = 5

-----------

## Load and process file with regular GLL grid

In [ ]:
regularGLL = np.loadtxt(fileDir+"regularGLL.txt")    # this file does not have repeated elements
sidx = [i for i in np.arange(0,len(regularGLL)-1,2)]
zidx = [i for i in np.arange(1,len(regularGLL),2)]
S = regularGLL[sidx]
Z = regularGLL[zidx]
nElem = len(S)                                       # total number of elements
S = S.reshape((S.shape[0]*S.shape[1],))
Z = Z.reshape((Z.shape[0]*Z.shape[1],))
S_Original = S.copy()
Z_Original = Z.copy()

## Coordinate transformation functions

In [ ]:
def sz2rtheta(s,z):
    '''
    Convert cylindrical s,z to spherical r, theta
    '''
    r = np.sqrt(s**2+z**2)
    theta = np.arccos(z/r)
    return r,theta

## Process file with Fourier coefficients

In [ ]:
if undulated: 
    fourier_order = int(Nr / 2 + 1)

    fcName = fileDir+"fCoeffs.txt" 
    undElemName = fileDir+"undulatedElementTags.txt" 
    with open(fcName) as f:             # This file only has the elements that are in the undulation range for one GEO model  
        lines1 = f.readlines()
    with open(undElemName) as f:        # These are the tag numbers for the elements in the above file
        lines2 = f.readlines()

    # These files contain the set of element twice, as models are built twice
    # Here we take the second half
    #lines2Cut = (int(re.sub(r"\D", "",lines2[-1]))-int(re.sub(r"\D", "",lines2[0]))+1)
    lines1Cut = int(len(lines1)/2)
    lines2Cut = int(len(lines2)/2)
    # lines1 =lines1[0:lines1Cut]
    # lines2 = lines2[0:lines2Cut]
    lines1 =lines1[lines1Cut:]
    lines2 = lines2[lines2Cut:]


    element_tag = []
    for i in range(0,len(lines2)):
        element_tag.append(int(re.sub(r"\D", "",lines2[i])))


    # convert the string-character format to only floats
    # structure of data is unchanged here
    # in each line, even index is real, odd index is imag
    # every 5 lines changes a fourier order
    # every 5*fourier_order lines changes an element
    fourier_coefficient = []
    for i in range(0,len(lines1)):
        lineList = lines1[i].replace(" ","").replace(")","").replace("\n","").split("(")[1:]
        oneLineFloat = []
        for k in range(0,5):
            onePointStr = lineList[k].split(",")
            onePointf1 = float(lineList[k].split(",")[0])
            onePointf2 = float(lineList[k].split(",")[1])
            oneLineFloat.append(onePointf1)
            oneLineFloat.append(onePointf2)

        fourier_coefficient.append(oneLineFloat)

    fourier_coefficient = np.array(fourier_coefficient)

    # line number ranges for m-th order for k-th element: np.arange(0,5)+int(5*(m+fourier_order * k))
    complex_type = np.complex32 if fourier_coefficient.dtype == np.complex64 else np.complex128

    phi_degree = 45
    phi = np.deg2rad(phi_degree)
    factor = 1.  # factor multiplied to dZ to amplify undulation for visulization purpose

    # Create a dictionary that stores dZ for all points in each element
    dict_data_dz = {}
    for k in range(0,len(element_tag)):
        # initialize undulation results with 0th order
        real_idx = [i for i in np.arange(0,10,2)]
        imag_idx = [i for i in np.arange(1,10,2)]
        dZ_inElem = np.reshape(fourier_coefficient[np.arange(0,5)+int(5*(0+fourier_order * k))][:,real_idx],-1)
        for m in range(1,int(fourier_order)):
            coeff_inElem = np.zeros((25,),dtype=complex_type)       
            coeff_inElem.real = np.reshape(fourier_coefficient[np.arange(0,5)+int(5*(m+fourier_order * k))][:,real_idx],-1)
            coeff_inElem.imag = np.reshape(fourier_coefficient[np.arange(0,5)+int(5*(m+fourier_order * k))][:,imag_idx],-1)        
            dZ_inElem += (2. * np.exp(1j * m * phi) * coeff_inElem).real
        key = element_tag[k]
        dict_data_dz[key] = dZ_inElem

        # now find idicies in the big flattened SZ coord array that correspond to these dZ's
        start_idx = element_tag[k] * 25
        idx_inElem = np.arange(start_idx,start_idx+25,1)
        # deform the mesh
        S_inElem = S[idx_inElem]
        Z_inElem = Z[idx_inElem]
        r,theta = sz2rtheta(S_inElem,Z_inElem)
        S[idx_inElem] += factor*dZ_inElem*np.sin(theta)
        Z[idx_inElem] += factor*dZ_inElem*np.cos(theta)
   

In [ ]:
np.cos(theta)

### Write dz to a pickle file for use with animation

In [ ]:
with open(fileDir+'dz_dict_'+modelName+'.pkl', "wb") as p:
    pickle.dump(dict_data_dz,p)

## Discontinuity lines
1. Surface
2. Moho

#### Find the GLL points that form these discontinuity lines
Read the following parameters from input and background model files

In [ ]:
def findDisconGLLsSpherical(disconStart, minV, maxColat, numElemPerWL, maxFreq, surface = False):
    
    minWL = minV/maxFreq
    horizElemLen = minWL/numElemPerWL
    earthRadius = 6371e3
    # number of elements on the surface arc, which is the same for deeper arcs unless there is mesh coarsening.
    numArcSurfElem = int(np.ceil(earthRadius * np.deg2rad(maxColat)/horizElemLen))
    print("Number of elements along arc:" + str(numArcSurfElem))
    
    disconZ = []
    disconS = []
    
    originalZ = []
    
    # a deeper discontinuity
    if ~surface:
        # Find the element tags of the elements whose bottom edge is the discontinuity
        disconElemTags = np.arange(disconStart,int(disconStart+numArcSurfElem))

        # Find the index for big Z array and extract the Z values that correspond to the bottom edge in these elements
        # 0 - 5 - 10 - 15 - 20 
        # but here we downsample as the output data:
        # 0 - 10 -20 
        for k in disconElemTags:
            start_idx = k * 25
            disconIDX_inElem = np.arange(start_idx,start_idx+21,10)
            disconZ.append(Z[disconIDX_inElem])
            disconS.append(S[disconIDX_inElem])
           
            originalZ.append(Z_Original[disconIDX_inElem])
    # the discontinuity is the surface
    else:
        disconElemTags = np.arange(disconStart,int(disconStart+numArcSurfElem))
        # Find the index for big Z array and extract the Z values that correspond to the bottom edge in these elements
        # 4 - 9 - 14 - 19 - 24
        # but here we downsample as the output data:
        # 4 - 14 -24 
        for k in disconElemTags:
            start_idx = k * 25
            disconIDX_inElem = np.arange(start_idx+4,start_idx+25,10)
            disconZ.append(Z[disconIDX_inElem])
            disconS.append(S[disconIDX_inElem])
            
            originalZ.append(Z_Original[disconIDX_inElem])
    
    disconZ = np.array(disconZ)
    disconZ = disconZ.reshape(-1)
    disconS = np.array(disconS)
    disconS = disconS.reshape(-1)

    originalZ = np.array(originalZ)
    originalZ = originalZ.reshape(-1)
    
    return disconZ, disconS,originalZ
        
        

### Moho

In [ ]:
disconStart =2116400
# This number selects which discontinuity to plot
# To find the above number, go to SE_Model.cpp in the dev version. This has to be done once for each discontinuity
# Don't forget to rebuild the code once discon depth is changed
######################
minV = 3200          # From background model. Exclude 0. m/s
maxColat = 18.2       # in degrees
numElemPerWL = 2.78
maxFreq = 5       # Hz
surface = False

disconZ, disconS, originalZ = findDisconGLLsSpherical(disconStart,
                                                      minV, 
                                                      maxColat, 
                                                      numElemPerWL, 
                                                      maxFreq,
                                                      surface)

In [ ]:
d = 100e3  # forward shift distance. This is used to shift boundaries to NOT conincide with point-cloud spatially
           # in paraview. Otherwise, Paraview can't render lines. 
           # Only needed if only 1 point per element is output.
           # If multiple points are output and mesh is represented by a surface but not point-cloud, then don't
           # need this. 

#+d*np.sin(np.deg2rad(phi_degree))
#*np.cos(np.deg2rad(phi_degree))
x = disconS*np.cos(np.deg2rad(phi_degree))+d*np.cos(np.deg2rad(phi_degree))
y = disconS*np.sin(np.deg2rad(phi_degree))+(-d)*np.sin(np.deg2rad(phi_degree))
z = disconZ
points = np.column_stack((x,y,z))

disconVTK = destDir + ""


# Make a spline by interpolation
moho = pv.Spline(points,400)
# Check if spline makes sense
#moho.plot(line_width=4, color="k")
# Save VTK file
moho.save(disconVTK,binary=True)

### LVZ Top

In [ ]:
disconStart =1249600
numElemPerWL /= 2
disconZ, disconS, originalZ = findDisconGLLsSpherical(disconStart,
                                                      minV, 
                                                      maxColat, 
                                                      numElemPerWL, 
                                                      maxFreq,
                                                      surface)

In [ ]:
x = disconS*np.cos(np.deg2rad(phi_degree)) +d*np.sin(np.deg2rad(phi_degree))
y = disconS*np.sin(np.deg2rad(phi_degree)) +(-d)*np.cos(np.deg2rad(phi_degree))
z = disconZ
points = np.column_stack((x,y,z))


disconVTK = destDir + ".vtk"


# # Make a spline by interpolation
lvzTop = pv.Spline(points,400)
# Check if spline makes sense
#moho.plot(line_width=4, color="k")
# Save VTK file
lvzTop.save(disconVTK,binary=True)

### LVZ Bot

In [ ]:
disconStart =286000

disconZ, disconS, originalZ = findDisconGLLsSpherical(disconStart,
                                                      minV, 
                                                      maxColat, 
                                                      numElemPerWL, 
                                                      maxFreq,
                                                      surface)

In [ ]:
x = disconS*np.cos(np.deg2rad(phi_degree)) +d*np.sin(np.deg2rad(phi_degree))
y = disconS*np.sin(np.deg2rad(phi_degree)) +(-d)*np.cos(np.deg2rad(phi_degree))
z = disconZ
points = np.column_stack((x,y,z))

disconVTK = destDir + ".vtk"


# Make a spline by interpolation
lvzBot = pv.Spline(points,400)
# Check if spline makes sense
#moho.plot(line_width=4, color="k")
# Save VTK file
lvzBot.save(disconVTK,binary=True)